<a href="https://colab.research.google.com/github/SyedLord/google_colab/blob/main/Files_Copy(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Files Copy**

In [ ]:
#@markdown <img src='https://www.logo.wine/a/logo/OneDrive/OneDrive-Logo.wine.svg' height="50" alt="onedrive-logo"/>
#@markdown <h1> <strong> Mount OneDrive

import os
import subprocess
import re
from google.colab import drive

# --- Step 1: Rclone aur dependencies install karein ---
print("⚙️ Installing Dependencies...")
os.system("apt-get -y install fuse3 > /dev/null 2>&1")

print("🔍 Fetching the latest rclone version...")
try:
    version_info = subprocess.run(["curl", "-s", "https://downloads.rclone.org/version.txt"], capture_output=True, text=True, check=True).stdout
    latest_version_match = re.search(r"rclone v([\d.]+)", version_info)
    latest_version = latest_version_match.group(1)
    print(f"✅ Latest rclone version found: v{latest_version}")

    print(f"📥 Installing rclone v{latest_version}...")
    os.system(f"wget -q https://downloads.rclone.org/v{latest_version}/rclone-v{latest_version}-linux-amd64.deb")
    os.system(f"apt install -y ./rclone-v{latest_version}-linux-amd64.deb > /dev/null 2>&1")
    os.system("rm rclone-v*-linux-amd64.deb") # Clean up the downloaded file
    print("✅ Rclone installed successfully.")
    os.system("rclone version")
except Exception as e:
    print(f"❌ Failed to install rclone. Error: {e}")

# --- Step 2: Google Drive ko Colab se connect karein ---
print("\n🔌 Mounting Google Drive...")
drive.mount('/content/drive')
print("✅ Google Drive mounted successfully.")

# --- Step 3: Rclone config file ko copy karein ---
# Source: Aapke Google Drive ka path
config_source_path = "/content/drive/MyDrive/Rclone/rclone.conf"
# Destination: Colab mein rclone ki default location
config_dest_path = "/root/.config/rclone/rclone.conf"

print("\n🔄 Setting up Rclone configuration...")
if os.path.exists(config_source_path):
    # Destination directory banayein (agar maujood nahi hai)
    os.makedirs(os.path.dirname(config_dest_path), exist_ok=True)
    # File ko copy karein
    os.system(f"cp '{config_source_path}' '{config_dest_path}'")
    print("✅ Rclone config file successfully copied.")
else:
    print(f"⚠️ Warning: Rclone config file not found at '{config_source_path}'.")

# --- Step 4: Rclone remote ko mount karein ---
drive_name = "onedrive"

mount_point = f"/content/{drive_name}"

print(f"\n🚀 Mounting '{drive_name}' at '{mount_point}'...")
os.makedirs(mount_point, exist_ok=True)

# rclone mount command
os.system(f"nohup rclone --vfs-cache-mode writes mount {drive_name}: {mount_point} &")

print(f"✅ Drive '{drive_name}' successfully mounted at '{mount_point}'.")

# Aap is command ko ek alag cell mein run karke check kar sakte hain ke mount hua ya nahi:
# !ls /content/onedrive

In [ ]:
#@markdown <img src='https://www.logo.wine/a/logo/OneDrive/OneDrive-Logo.wine.svg' height="50" alt="onedrive-logo"/>
#@markdown <h1> <strong> Mount Auto-Detected Remotes

import os
import subprocess
import re
from google.colab import drive

# --- Step 1: Rclone aur dependencies install karein ---
print("⚙️ Installing Dependencies...")
os.system("apt-get -y install fuse3 > /dev/null 2>&1")

print("🔍 Fetching the latest rclone version...")
try:
    version_info = subprocess.run(["curl", "-s", "https://downloads.rclone.org/version.txt"], capture_output=True, text=True, check=True).stdout
    latest_version_match = re.search(r"rclone v([\d.]+)", version_info)
    latest_version = latest_version_match.group(1)
    print(f"✅ Latest rclone version found: v{latest_version}")

    print(f"📥 Installing rclone v{latest_version}...")
    os.system(f"wget -q https://downloads.rclone.org/v{latest_version}/rclone-v{latest_version}-linux-amd64.deb")
    os.system(f"apt install -y ./rclone-v{latest_version}-linux-amd64.deb > /dev/null 2>&1")
    os.system("rm rclone-v*-linux-amd64.deb")
    print("✅ Rclone installed successfully.")
except Exception as e:
    print(f"❌ Failed to install rclone. Error: {e}")

# --- Step 2: Google Drive ko Colab se connect karein ---
print("\n🔌 Mounting Google Drive to access config...")
drive.mount('/content/drive')

# --- Step 3: Config File Setup & Backup Check ---
config_source_path = "/content/drive/MyDrive/Rclone/rclone.conf"
config_dest_path = "/root/.config/rclone/rclone.conf"

print("\n🔄 Checking Rclone configuration...")

# Case A: Agar Colab mein config pehle se hai (shayad tumne abhi edit ki ho), to usay Drive pe backup karo
if os.path.exists(config_dest_path):
    print("ℹ️ Local config found. Syncing changes BACK to Google Drive (Safety Backup)...")
    try:
        os.system(f"cp '{config_dest_path}' '{config_source_path}'")
        print("✅ Config file backed up to Google Drive successfully.")
    except Exception as e:
        print(f"⚠️ Could not backup config to Drive: {e}")

# Case B: Agar Colab mein nahi hai, to Drive se le aao
elif os.path.exists(config_source_path):
    os.makedirs(os.path.dirname(config_dest_path), exist_ok=True)
    os.system(f"cp '{config_source_path}' '{config_dest_path}'")
    print("✅ Rclone config file copied from Google Drive.")
else:
    print(f"⚠️ Warning: Rclone config file not found at '{config_source_path}'. Please run '!rclone config' first.")

# --- Step 4: Auto-Detect & Mount All Remotes ---
print("\n🔎 Detecting configured remotes...")

try:
    # rclone listremotes command se saare naam nikaal rahe hain
    result = subprocess.check_output(["rclone", "listremotes"], text=True)
    remotes = result.strip().split("\n")

    # Empty lines remove karein
    remotes = [r.strip() for r in remotes if r.strip()]

    if not remotes:
        print("⚠️ No remotes found in config! Please add one using '!rclone config'.")
    else:
        print(f"✅ Found {len(remotes)} remotes: {', '.join(remotes)}")

        for remote in remotes:
            # Remote name se ':' hatana zaroori hai folder name ke liye
            drive_name = remote.replace(":", "")
            mount_point = f"/content/{drive_name}"

            print(f"\n🚀 Mounting '{drive_name}' at '{mount_point}'...")
            os.makedirs(mount_point, exist_ok=True)

            # Mount command
            os.system(f"nohup rclone --vfs-cache-mode writes mount {remote} {mount_point} &")
            print(f"✅ Mount command sent for '{drive_name}'.")

except subprocess.CalledProcessError:
    print("❌ Error: Could not list remotes. Is the config file valid?")

In [ ]:
import os
import shutil
import sys
from tqdm.notebook import tqdm
import time
from datetime import datetime
from openpyxl import Workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font, PatternFill

# Global tracking list
file_log = []
total_transferred_size = 0
skipped_files = 0
start_time = time.time()

report_path = f"/content/transfer_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"

def get_file_size_str(size_in_bytes):
    for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
        if size_in_bytes < 1024.0 or unit == 'TB':
            return f"{size_in_bytes:.2f} {unit}"
        size_in_bytes /= 1024.0

def log_file(src, dst, size, status):
    global total_transferred_size
    total_transferred_size += size
    file_log.append([
        os.path.basename(src),
        get_file_size_str(size),
        status,
        src,
        dst
    ])

def copy_with_progress(src_path, dst_path):
    file_size = os.path.getsize(src_path)
    file_size_str = get_file_size_str(file_size)
    os.makedirs(os.path.dirname(dst_path), exist_ok=True)

    with open(src_path, 'rb') as fsrc:
        with open(dst_path, 'wb') as fdst:
            with tqdm(total=file_size, unit='B', unit_scale=True,
                      desc=f"Copying {os.path.basename(src_path)} ({file_size_str})",
                      bar_format='{desc}: {percentage:3.0f}%|{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}]') as pbar:
                chunk_size = 1024 * 1024
                while True:
                    chunk = fsrc.read(chunk_size)
                    if not chunk:
                        break
                    fdst.write(chunk)
                    pbar.update(len(chunk))
    log_file(src_path, dst_path, file_size, "Copied")

def move_with_progress(src_path, dst_path):
    copy_with_progress(src_path, dst_path)
    os.remove(src_path)
    print(f"Moved: {os.path.basename(src_path)} -> {dst_path}")
    file_log[-1][2] = "Moved"

def get_disk_space(path):
    usage = shutil.disk_usage(path)
    return get_file_size_str(usage.free), get_file_size_str(usage.total)

def write_excel_report():
    wb = Workbook()
    ws = wb.active
    ws.title = "Transfer Report"

    time_taken = time.time() - start_time

    bold_font = Font(bold=True)
    header_fill = PatternFill(start_color="FFC000", end_color="FFC000", fill_type="solid")

    summary_data = [
        ["Summary"],
        ["Total Files", len(file_log)],
        ["Total Size Transferred", get_file_size_str(total_transferred_size)],
        ["Total Size Moved", get_file_size_str(total_transferred_size) if operation == 'move' else "0.00 B"],
        ["Total Size Copied", get_file_size_str(total_transferred_size) if operation == 'copy' else "0.00 B"],
        ["Skipped Files", skipped_files],
        ["Time Taken", f"{time_taken:.2f} seconds"],
        []
    ]

    for i, row in enumerate(summary_data):
        ws.append(row)
        if i == 0:
            for cell in ws[ws.max_row]:
                cell.font = bold_font
                cell.fill = header_fill

    headers = ["Filename", "Size", "Status", "Source Path", "Destination Path"]
    ws.append(headers)
    for cell in ws[ws.max_row]:
        cell.font = bold_font
        cell.fill = header_fill

    for row in file_log:
        ws.append(row)

    for col in ws.columns:
        max_length = max(len(str(cell.value)) if cell.value else 0 for cell in col)
        col_letter = get_column_letter(col[0].column)
        ws.column_dimensions[col_letter].width = max_length + 2

    wb.save(report_path)
    print(f"\nExcel report saved to: {report_path}\n")

def process_path(src, dst, operation='copy', create_parent_folder=False):
    global src_path, dst_path
    src_path, dst_path = src, dst

    if not os.path.exists(src):
        print(f"Source path '{src}' does not exist.")
        return False

    # If enabled, add the parent folder name to destination
    if create_parent_folder and os.path.isdir(src):
        parent_folder_name = os.path.basename(os.path.normpath(src))
        dst = os.path.join(dst, parent_folder_name)

    if os.path.isfile(src):
        dst_file_path = os.path.join(dst, os.path.basename(src)) if os.path.isdir(dst) else dst
        os.makedirs(os.path.dirname(dst_file_path), exist_ok=True)
        print(f"Processing single file: {src}")
        move_with_progress(src, dst_file_path) if operation == 'move' else copy_with_progress(src, dst_file_path)
        write_excel_report()
        return True

    elif os.path.isdir(src):
        total_files = sum([len(files) for _, _, files in os.walk(src)])
        print(f"Found {total_files} files to {operation}")
        if total_files == 0:
            print(f"No files found in directory '{src}'.")
            return False

        for root, _, files in os.walk(src):
            for file in files:
                try:
                    src_file_path = os.path.join(root, file)
                    rel_path = os.path.relpath(root, src)
                    dst_subdir = os.path.join(dst, rel_path)
                    os.makedirs(dst_subdir, exist_ok=True)
                    dst_file_path = os.path.join(dst_subdir, file)
                    move_with_progress(src_file_path, dst_file_path) if operation == 'move' else copy_with_progress(src_file_path, dst_file_path)
                except Exception as e:
                    print(f"Skipped file {file}: {e}")
                    global skipped_files
                    skipped_files += 1

        if operation == 'move':
            for root, dirs, files in os.walk(src, topdown=False):
                for dir in dirs:
                    try: os.rmdir(os.path.join(root, dir))
                    except OSError: pass
            try: os.rmdir(src)
            except OSError: pass

        write_excel_report()
        return True

    else:
        print(f"Source path '{src}' is neither a file nor a directory.")
        return False

if __name__ == "__main__":
    src_path = '' #@param {type:"string"}
    dst_path = '' #@param {type:"string"}
    operation = 'copy' #@param ['copy', 'move']
    create_parent_folder = False #@param {type:"boolean"}

    process_path(src_path, dst_path, operation, create_parent_folder)


## <img src='https://upload.wikimedia.org/wikipedia/commons/thumb/e/e4/Rclone_wide_logo.svg/1920px-Rclone_wide_logo.svg.png' height="50" alt="rclone-logo"/>
##**Mount Other Drives to Google Colab**

In [ ]:
#@markdown <center> <strong> <h1> ⚙️ RUN THIS CELL TO INSTALL DEPENDENCIES ⚙️ </strong> </center>
import os
import subprocess
import re

# Install fuse3 (if not already installed)
print("Installing fuse3...")
os.system("apt-get -y install fuse3 > /dev/null 2>&1")

# Fetch the latest rclone version dynamically
print("Fetching the latest rclone version...")
version_info = subprocess.run(["curl", "-s", "https://downloads.rclone.org/version.txt"], capture_output=True, text=True).stdout
latest_version_match = re.search(r"rclone v([\d.]+)", version_info)

if latest_version_match:
    latest_version = latest_version_match.group(1)
    print(f"Latest rclone version found: {latest_version}")
else:
    print("Failed to fetch the latest rclone version.")
    exit()

# Download and install the latest rclone version
print(f"Installing rclone v{latest_version}...")
os.system(f"wget -q https://downloads.rclone.org/v{latest_version}/rclone-v{latest_version}-linux-amd64.deb")
os.system(f"apt install -y ./rclone-v{latest_version}-linux-amd64.deb > /dev/null 2>&1")

# Verify the installation
print("Verifying rclone installation...")
os.system("rclone version")

In [ ]:
#@markdown <h1><strong><center> Authenticate 🔐
!rclone config

In [ ]:
#@markdown <h1> <strong> <center> Mount Drive 🔥
import os

# User input for the drive name (Google Colab's @param style)
drive_name = ""  #@param {type:"string"}

# Create mount point dynamically
mount_point = f"/content/{drive_name}"
os.system(f"mkdir -p {mount_point}")

# Mount the drive dynamically using the user-defined name
os.system(f"nohup rclone --vfs-cache-mode writes mount {drive_name}: {mount_point} &")

print(f"✅ Drive '{drive_name}' successfully mounted at '{mount_point}'.")
